In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/drw-crypto-market-prediction/sample_submission.csv
/kaggle/input/drw-crypto-market-prediction/train.parquet
/kaggle/input/drw-crypto-market-prediction/test.parquet


# DRW - Crypto Market Prediction

This notebook documents all the steps done in this project.

Timeline:
- 10/06/25: 0.05031
    - Reorganize notebooks.
    - Test training with GPU - Way faster than CPU.
    - Implement feature elimination using GPU.
    - Tested with Linear Models - will be extremely slow in iteration.
- 

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from scipy.stats import pearsonr
import polars as pl
import numpy as np
from tqdm import tqdm
from datetime import datetime

# 1 Data

In [3]:
data = pl.read_parquet(
    "/kaggle/input/drw-crypto-market-prediction/train.parquet"
).sort("timestamp", descending = False)
data

bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,…,X856,X857,X858,X859,X860,X861,X862,X863,X864,X865,X866,X867,X868,X869,X870,X871,X872,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label,timestamp
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns]
15.283,8.425,176.405,44.984,221.389,0.121263,-0.41769,0.005399,0.125948,0.058359,0.027359,0.03578,0.068219,1.034825,-0.029575,0.327805,0.485823,0.668596,0.617389,0.770037,0.857631,1.754456,0.572503,0.883229,0.58567,0.816321,0.529973,0.508244,0.448616,1.341892,1.406392,0.953631,1.183991,1.474789,0.774389,0.660586,0.269043,…,-0.216525,0.200508,0.492433,-0.51249,0.541286,-0.336399,-1.027483,0.21857,0.0,1.728155,0.62414,0.0,-0.051211,0.0,0.0,0.0,0.0,0.691754,0.242124,2.096157,3.369195,0.244667,0.286611,0.722679,0.901931,1.000007,1.925423,1.847943,0.005676,0.190791,0.369691,0.37763,0.210153,0.159183,0.530636,0.562539,2023-03-01 00:00:00
38.59,2.336,525.846,321.95,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,0.208359,0.217057,0.249624,0.948694,-0.183488,0.150526,0.308421,0.492232,0.529787,0.682958,0.770965,1.686504,0.273357,0.591695,0.442391,0.674792,0.460741,0.439681,0.380399,1.304113,1.003783,0.776628,1.015943,1.312735,0.696895,0.584217,0.231104,…,-0.180112,0.213252,0.479806,-0.180527,0.450331,-0.31915,-1.024055,0.088014,0.0,1.665698,0.622775,0.0,-0.079621,0.0,0.0,0.0,0.0,0.691665,0.242091,2.46103,4.127584,0.321394,0.31246,0.746452,0.912371,1.003153,1.928569,1.849468,0.005227,0.18466,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686,2023-03-01 00:01:00
0.442,60.25,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,0.072778,0.081564,0.114166,0.896459,-0.261779,0.044571,0.200608,0.384558,0.476229,0.629848,0.718232,1.656707,0.140156,0.457268,0.376524,0.610116,0.429751,0.409316,0.350359,1.28325,0.760801,0.670816,0.917205,1.219124,0.653355,0.541739,0.210095,…,-0.265966,0.191734,0.440207,-0.108209,0.420681,-0.316953,-1.024056,-0.147363,0.0,1.666893,0.621414,0.0,-0.080427,0.0,0.0,0.0,0.0,0.691674,0.242093,2.493249,4.182112,0.326701,0.314636,0.746681,0.911129,1.002502,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505,2023-03-01 00:02:00
4.865,21.016,335.742,124.963,460.705,0.072944,-0.43659,-0.102483,0.017551,0.007149,-0.021681,-0.012936,0.019634,0.732634,-0.535845,-0.273947,-0.124959,0.056438,0.311539,0.465377,0.554022,1.663491,0.152084,0.468778,0.383696,0.618529,0.435326,0.415523,0.356895,1.319538,0.955549,0.789646,1.044941,1.353001,0.72392,0.613462,0.246212,…,-0.322244,0.183687,0.404295,-0.169373,0.386584,-0.314775,-1.024058,-0.09459,0.0,1.735322,0.620057,0.0,-0.094702,0.0,0.0,0.0,0.0,0.69121,0.24193,2.525526,4.292975,0.350791,0.32357,0.753829,0.913363,1.002985,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703,2023-03-01 00:03:00
27.158,3.451,98.411,44.407,142.818,0.17382,-0.213489,0.096067,0.215709,0.107133,0.078976,0.087818,0.120426,0.763537,-0.430945,-0.205298,-0.062118,0.117266,0.341493,0.495591,0.584519,1.668419,0.156177,0.472732,0.3871,0.623192,0.439034,0.419868,0.361572,1.324595,0.90546,0.78375,1.047708,1.36188,0.732001,0.622712,0.251095,…,-0.369625,0.192377,0.415438,-0.198976,0.389969,-0.312628,-1.02406,0.162221,0.0,1.712096,0.618703,0.0,-0.091884,0.0,0.0,0.0,0.0,0.691207,0.241928,2.52443,4.306694,0.335599,0.31907,0.747533,0.908904,1.001286,1.927084,1.84895,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452,2023-03-01 00:04:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

## 1.1 Pre-processing / Feature Engineering

**Pre-Processing**
1. inf/-inf columns: `['X697', 'X698', 'X699', 'X700', 'X701', 'X702', 'X703', 'X704', 'X705', 'X706', 'X707', 'X708', 'X709', 'X710', 'X711', 'X712', 'X713', 'X714', 'X715', 'X716', 'X717']`
2. columns with NaN values: `[]`
3. 0 std columns : `['X864', 'X867', 'X869', 'X870', 'X871', 'X872']`


**Feature Engineering**
1. `bidask_ratio`
2. `buysell_ratio`
3. `bidask_delta`
4. `buysell_delta`
5. `buysell_size`
6. `bidask_size`

In [4]:
def get_cols_inf(df: pl.DataFrame) -> list[str]:
    """
    Returns a list of column names that contain any positive or negative infinity.
    """
    cols = []
    for col in df.columns:
        # df[col] is a Series; .is_infinite() → Boolean Series; .any() → Python bool
        try:
            if df[col].is_infinite().any():
                cols.append(col)
        except Exception:
            # if the column isn’t numeric, .is_infinite() might error—just skip it
            continue
    return cols

def get_nan_columns(df: pl.DataFrame) -> list[str]:
    """
    Returns a list of column names with any NaN/null values.
    """
    cols = []
    for col in df.columns:
        if df.select(pl.col(col).is_null().any()).item():
            cols.append(col)
    return cols

def get_cols_zerostd(df: pl.DataFrame) -> list[str]:
    """
    Returns a list of column names whose standard deviation is zero
    (or whose std returns None because all values are null).
    Non-numeric columns (e.g. datetime) are skipped.
    """
    cols = []
    for col, dtype in zip(df.columns, df.dtypes):
        # Only attempt std() on numeric dtypes
        if dtype.is_numeric():  
            # df[col] is a Series; .std() returns a Python float or None
            std_val = df[col].std()
            if std_val == 0.0 or std_val is None:
                cols.append(col)
    return cols


def feature_engineering(df: pl.DataFrame) -> pl.DataFrame:
    # Feature engineering
    df = df.with_columns([
        # bidask_ratio = bid_qty / ask_qty
        (pl.col("bid_qty") / pl.col("ask_qty")).alias("bidask_ratio"),

        # buysell_ratio = 0 if volume == 0 else buy_qty / sell_qty
        pl.when(pl.col("volume") == 0)
        .then(0)
        .otherwise(pl.col("buy_qty") / pl.col("sell_qty"))
        .alias("buysell_ratio"),

        # bidask_delta = bid_qty - ask_qty
        (pl.col("bid_qty") - pl.col("ask_qty")).alias("bidask_delta"),

        # buysell_delta = buy_qty - sell_qty
        (pl.col("buy_qty") - pl.col("sell_qty")).alias("buysell_delta"),

        # buysell_size = buy_qty + sell_qty
        (pl.col("buy_qty") + pl.col("sell_qty")).alias("buysell_size"),

        # bidask_size = bid_qty + ask_qty
        (pl.col("bid_qty") + pl.col("ask_qty")).alias("bidask_size"),
    ])
    return df
def preprocess_train(train: pl.DataFrame, columns_to_drop: list[str] = []) -> pl.DataFrame:
    """
    Mirror of the original pandas workflow, but using polars.
    1. Identify columns with infinite, NaN, or zero‐std and drop them.
    2. Drop any user‐specified columns (e.g. label or order‐book columns).
    3. (You can add normalized/scaling steps here if needed.)
    """
    df = train.clone()

    df = feature_engineering(df)
    
    #### Preprocessing
    cols_inf = get_cols_inf(df)
    print("Columns with infinite values:", cols_inf)

    cols_nan = get_nan_columns(df)
    print("Columns with NaN values:", cols_nan)

    cols_zerostd = get_cols_zerostd(df)
    print("Columns with zero standard deviation:", cols_zerostd)
    # Drop columns with infinite, NaN, or zero‐std values
    drop_columns = list(set(cols_inf) | set(cols_nan) | set(cols_zerostd) | set(columns_to_drop))
    if drop_columns:
        df = df.drop(drop_columns)
    # df = df.sort("timestamp", descending=False)
    return df, drop_columns

def preprocess_test(test: pl.DataFrame, columns_to_drop: list[str] = []) -> pl.DataFrame:
    df = test.clone()
    df = feature_engineering(df)
    df = df.drop(columns_to_drop)
    print("Columns dropped from test set:", columns_to_drop)
    return df

In [5]:
y = data["label"]
X, drop_columns = preprocess_train(
    data,
    columns_to_drop=["label", "bid_qty", "ask_qty", "buy_qty", "sell_qty"]
)
X

Columns with infinite values: ['X697', 'X698', 'X699', 'X700', 'X701', 'X702', 'X703', 'X704', 'X705', 'X706', 'X707', 'X708', 'X709', 'X710', 'X711', 'X712', 'X713', 'X714', 'X715', 'X716', 'X717']
Columns with NaN values: []
Columns with zero standard deviation: ['X864', 'X867', 'X869', 'X870', 'X871', 'X872']


volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,…,X855,X856,X857,X858,X859,X860,X861,X862,X863,X865,X866,X868,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,timestamp,bidask_ratio,buysell_ratio,bidask_delta,buysell_delta,buysell_size,bidask_size
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],f64,f64,f64,f64,f64,f64
221.389,0.121263,-0.41769,0.005399,0.125948,0.058359,0.027359,0.03578,0.068219,1.034825,-0.029575,0.327805,0.485823,0.668596,0.617389,0.770037,0.857631,1.754456,0.572503,0.883229,0.58567,0.816321,0.529973,0.508244,0.448616,1.341892,1.406392,0.953631,1.183991,1.474789,0.774389,0.660586,0.269043,0.962802,0.966513,0.952759,0.952916,…,0.418618,-0.216525,0.200508,0.492433,-0.51249,0.541286,-0.336399,-1.027483,0.21857,1.728155,0.62414,-0.051211,0.691754,0.242124,2.096157,3.369195,0.244667,0.286611,0.722679,0.901931,1.000007,1.925423,1.847943,0.005676,0.190791,0.369691,0.37763,0.210153,0.159183,0.530636,2023-03-01 00:00:00,1.814006,3.921505,6.858,131.421,221.389,23.708
847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,0.208359,0.217057,0.249624,0.948694,-0.183488,0.150526,0.308421,0.492232,0.529787,0.682958,0.770965,1.686504,0.273357,0.591695,0.442391,0.674792,0.460741,0.439681,0.380399,1.304113,1.003783,0.776628,1.015943,1.312735,0.696895,0.584217,0.231104,0.935145,0.938957,0.918275,0.919065,…,0.424977,-0.180112,0.213252,0.479806,-0.180527,0.450331,-0.31915,-1.024055,0.088014,1.665698,0.622775,-0.079621,0.691665,0.242091,2.46103,4.127584,0.321394,0.31246,0.746452,0.912371,1.003153,1.928569,1.849468,0.005227,0.18466,0.363642,0.374515,0.209573,0.158963,0.530269,2023-03-01 00:01:00,16.519692,1.633316,36.254,203.896,847.796,40.926
295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,0.072778,0.081564,0.114166,0.896459,-0.261779,0.044571,0.200608,0.384558,0.476229,0.629848,0.718232,1.656707,0.140156,0.457268,0.376524,0.610116,0.429751,0.409316,0.350359,1.28325,0.760801,0.670816,0.917205,1.219124,0.653355,0.541739,0.210095,0.932614,0.936476,0.919497,0.92028,…,0.409942,-0.265966,0.191734,0.440207,-0.108209,0.420681,-0.316953,-1.024056,-0.147363,1.666893,0.621414,-0.080427,0.691674,0.242093,2.493249,4.182112,0.326701,0.314636,0.746681,0.911129,1.002502,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,2023-03-01 00:02:00,0.007336,1.167619,-59.808,22.858,295.596,60.692
460.705,0.072944,-0.43659,-0.102483,0.017551,0.007149,-0.021681,-0.012936,0.019634,0.732634,-0.535845,-0.273947,-0.124959,0.056438,0.311539,0.465377,0.554022,1.663491,0.152084,0.468778,0.383696,0.618529,0.435326,0.415523,0.356895,1.319538,0.955549,0.789646,1.044941,1.353001,0.72392,0.613462,0.246212,0.936911,0.942204,0.940304,0.942497,…,0.400075,-0.322244,0.183687,0.404295,-0.169373,0.386584,-0.314775,-1.024058,-0.09459,1.735322,0.620057,-0.094702,0.69121,0.24193,2.525526,4.292975,0.350791,0.32357,0.753829,0.913363,1.002985,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,2023-03-01 00:03:00,0.23149,2.686731,-16.151,210.779,460.705,25.881
142.818,0.17382,-0.213489,0.096067,0.215709,0.107133,0.078976,0.087818,0.120426,0.763537,-0.430945,-0.205298,-0.062118,0.117266,0.341493,0.495591,0.584519,1.668419,0.156177,0.472732,0.3871,0.623192,0.439034,0.419868,0.361572,1.324595,0.90546,0.78375,1.047708,1.36188,0.732001,0.622712,0.251095,0.931761,0.936818,0.928362,0.930464,…,0.391759,-0.369625,0.192377,0.415438,-0.198976,0.389969,-0.312628,-1.02406,0.162221,1.712096,0.618703,-0.091884,0.691207,0.241928,2.52443,4.306694,0.335599,0.31907,0.747533,0.908904,1.001286,1.927084,1.84895,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,202

# 1.2 Time Series Split Functions

1. `split_rollingwindow` - rolling window.
3. `split_overlapwindow` - overlapping.

In [6]:
def split_rollingwindow(X, n_splits=5, train_ratio=0.8):
    """
    Rolling window time series splitter with fixed train/test ratio and number of splits.
    
    Parameters
    ----------
    X : array-like or DataFrame
        Dataset with time-ordering preserved.
    n_splits : int
        Number of rolling splits.
    train_ratio : float
        Proportion of each window used for training (e.g. 0.8 for 80/20 split).
    
    Yields
    ------
    train_idx, test_idx : range, range
        Index ranges for training and testing.
    """
    n_obs = len(X)
    window_size = n_obs // (n_splits + 1)
    train_size = int(train_ratio * window_size)
    test_size = window_size - train_size
    
    for i in range(n_splits):
        start = i * window_size
        train_start = start
        train_end = train_start + train_size
        test_start = train_end
        test_end = test_start + test_size

        if test_end > n_obs:
            break

        yield range(train_start, train_end), range(test_start, test_end)

# splits = rolling_window_split(X=X, n_splits = 5, train_ratio=0.5)
# for train_idx, test_idx in splits:
#     print(train_idx, test_idx)
# #     df_train = X.slice(train_idx.start, len(train_idx))
# #     df_test = X.slice(test_idx.start, len(test_idx))

# for i, (train_idx, test_idx) in enumerate(splits):
#     if i == 1:  # second batch (index 1)
#         df_train = X.slice(train_idx.start, len(train_idx))
#         df_test = X.slice(test_idx.start, len(test_idx))
#         break

In [7]:
def split_overlapwindow(X, train_size, test_size, step):
    """
    Rolling window splitter with overlapping train/test splits.

    Parameters
    ----------
    X : array-like or DataFrame
        Dataset with time-ordering preserved.
    train_size : int
        Number of observations in each training window.
    test_size : int
        Number of observations in each test window.
    step : int
        Forward step size between each split.

    Yields
    ------
    train_idx, test_idx : range, range
        Index ranges for training and testing.
    """
    n_obs = len(X)
    start = 0
    while (start + train_size + test_size) <= n_obs:
        train_start = start
        train_end = train_start + train_size
        test_start = train_end
        test_end = test_start + test_size

        yield range(train_start, train_end), range(test_start, test_end)
        start += step

train_size = 100000
test_size = 100000
step = 10000
splits = split_overlapwindow(X, train_size, test_size, step)
for train_idx, test_idx in splits:
    print(train_idx, test_idx)

range(0, 100000) range(100000, 200000)
range(10000, 110000) range(110000, 210000)
range(20000, 120000) range(120000, 220000)
range(30000, 130000) range(130000, 230000)
range(40000, 140000) range(140000, 240000)
range(50000, 150000) range(150000, 250000)
range(60000, 160000) range(160000, 260000)
range(70000, 170000) range(170000, 270000)
range(80000, 180000) range(180000, 280000)
range(90000, 190000) range(190000, 290000)
range(100000, 200000) range(200000, 300000)
range(110000, 210000) range(210000, 310000)
range(120000, 220000) range(220000, 320000)
range(130000, 230000) range(230000, 330000)
range(140000, 240000) range(240000, 340000)
range(150000, 250000) range(250000, 350000)
range(160000, 260000) range(260000, 360000)
range(170000, 270000) range(270000, 370000)
range(180000, 280000) range(280000, 380000)
range(190000, 290000) range(290000, 390000)
range(200000, 300000) range(300000, 400000)
range(210000, 310000) range(310000, 410000)
range(220000, 320000) range(320000, 420000)
ra

# 2 Iterative Feature Selection

## 2.1 Base Model

In [8]:
from xgboost import XGBRegressor

# model = XGBRegressor(
#     tree_method="gpu_hist",
#     predictor="gpu_predictor",
#     n_estimators=100,
#     max_depth=8,
#     learning_rate=0.05,
#     random_state=42,
#     n_jobs=-1
# )

model = XGBRegressor(
    tree_method = "hist", 
    device = "cuda",
    n_estimators=100,
    max_depth=8,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)


# Convert your data to GPU arrays
# X_gpu = cp.asarray(X.to_numpy())
# y_gpu = cp.asarray(y.to_numpy())


## 2.2 Iteration

In [9]:
from sklearn.base import clone
from sklearn.metrics import mean_squared_error
import uuid
import cupy as cp  # You'll need cupy installed

# def iterative_featureselection_onesplit(model, X, y, split_fn, scoring, drop_fraction = 0.1, min_features = 10):
#     results = []
#     splits = list(split_fn(X))
#     n_splits = len(splits)

#     current_X = X.copy()
#     iteration = 0
    
#     while current_X.shape[1] > min_features:
#         iteration += 1
#         print(f"\n[+] Iteration {iteration} - {current_X.shape[1]} features")
        
#         training_model = clone(model)
#         training_model.fit(current_X, y)
#         importances = pd.Series(fitted_model.feature_importances_, index=current_X.columns)

#         scores = []
#         for i, (train_idx, test_idx) in enumerate(splits, start=1):
#             print(f"    [Fold {i}/{n_splits}] ...", end="\r")
            
            
#             X_train, y_train = current_X.iloc[train_idx], y.iloc[train_idx]
#             X_test, y_test = current_X.iloc[test_idx], y.iloc[test_idx]
        
#             fold_model = clone(model)
#             fold_model.fit(X_train, y_train)
        
#             preds = fold_model.predict(X_test)
#             score = scoring(y_test, preds)
#             scores.append(score)
        
#         mean_score = np.mean(score)

#         results.append({
#             "iteration" : iteration,
#             "model_name": model.__class__.__name__
#             "num_features": current_X.shape[1],
#             "features": current_X.columns.tolist(),
#             "score_mean": mean_score,
#         })
#         n_drop = max(10, int(len(importances) * drop_fraction))
#         drop_cols = importances.nsmallest(n_drop).index
#         current_X = current_X.drop(columns=drop_cols)

#     print(f"\n[*] Finished: {current_X.shape[1]} features remaining (≤ min_features={min_features})")
#     return results

# results = iterative_featureselection(
#     model=model,
#     X=X,
#     y=y,
#     split_fn=lambda X: rolling_window_split(X, train_size=100_000, test_size=20_000, step=20_000),
#     drop_fraction=0.1,
#     min_features=50,
#     scoring=pearson_corr
# )

def iterative_featureselection(
    model,
    X: pl.DataFrame,
    y: pl.Series,
    split_fn,
    scorers: dict,
    drop_fraction: float = 0.1,
    min_features: int = 10,
    use_shap: bool = False,
    use_gpu : bool = True
) -> list:
    ID = uuid.uuid4().hex
    results = []
    splits = list(split_fn(X))
    n_splits = len(splits)

    current_X = X.clone()
    feature_cols = current_X.columns
    iteration = 0

    # Convert y to appropriate format
    if use_gpu:
        try:
            import cupy as cp
            y_np = cp.asarray(y.to_numpy().flatten())
        except ImportError:
            print("CuPy not available, falling back to CPU")
            y_np = y.to_numpy().flatten()
            use_gpu = False

    while len(feature_cols) > min_features:
        iteration += 1
        print(f"\n[+] Iteration {iteration} - {len(feature_cols)} features")

        # Extract feature matrix
        X_np = current_X.select(feature_cols).to_numpy()
        if use_gpu:
            X_np = cp.asarray(X_np)
            
        # Fit full model for feature importances
        training_model = clone(model)
        training_model.fit(X_np, y_np)

        # 1. Compute raw importances as a NumPy array
        if use_shap:
            explainer   = shap.TreeExplainer(training_model)
            shap_vals   = explainer.shap_values(X_np)          # shape (n_samples, n_features)
            imp_array   = np.abs(shap_vals).mean(axis=0)        # mean(|SHAP|) per feature
        else:
            imp_array   = np.array(training_model.feature_importances_)
        
        # 2. Build a Polars DataFrame of (feature, importance)
        importances_df = pl.DataFrame({
            "feature": feature_cols,
            "importance": imp_array.tolist()
        })
        
        # Cross-validation
        metrics = {k: [] for k in scorers}

        for i, (train_idx, test_idx) in enumerate(splits, start=1):
            print(f"    [Fold {i}/{n_splits}] ...", end="\r")

            X_train, y_train = X_np[train_idx], y_np[train_idx]
            X_test, y_test = X_np[test_idx], y_np[test_idx]

            fold_model = clone(model)
            fold_model.fit(X_train, y_train)
            y_pred = fold_model.predict(X_test)

            for name, func in scorers.items():
                y_test = y_test.get() if hasattr(y_test, 'get') else y_test
                y_pred = y_pred.get() if hasattr(y_pred, 'get') else y_pred
                metrics[name].append(func(y_test, y_pred))

        # Mean metric values
        mean_metrics = {name: np.mean(vals) for name, vals in metrics.items()}

        results.append({
            "ID": ID,
            "params": {
                "drop_fraction" : drop_fraction,
                "min_features": min_features,
                "use_shap" : use_shap,
                "use_gpu" : use_gpu
            },
            "num_features": len(feature_cols),
            "features": feature_cols.copy(),
            **{f"scores_{name}_mean": val for name, val in mean_metrics.items()}
        })

        print("    → " + " | ".join([f"{k.upper()}: {v:.4f}" for k, v in mean_metrics.items()]))

        # 3. Identify the n_drop least important features
        n_drop    = max(10, int(len(feature_cols) * drop_fraction))
        drop_cols = (
            importances_df
            .sort("importance")        # ascending
            .head(n_drop)              # take the smallest n_drop
            .get_column("feature")     # extract the feature column
            .to_list()                 # into a Python list for filtering
        )
        
        # 4. Filter out dropped features in your Polars DataFrame
        feature_cols = [f for f in feature_cols if f not in drop_cols]
        current_X    = current_X.select(feature_cols)

    print(f"""\n {"["+ "*" * 10 + "]"} Finished: {len(feature_cols)} features remaining (≤ min_features={min_features}) {"["+ "*" * 10 + "]"}""")
    return results
    
def pearson_corr(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

scorers = {
    "pearson": pearson_corr,
    "mse": mean_squared_error,
    "mae": mean_absolute_error
}

# Test
# results = iterative_featureselection(
#     model=model,
#     X=X,
#     y=y,
#     split_fn=lambda X:split_rollingwindow(X=X, n_splits = 2, train_ratio=0.5),
#     scorers = scorers,
#     drop_fraction=0.9,
#     min_features=100,
#     use_shap = False
# )

In [10]:
import json

def save_results_to_json(results, out_path):
    """Save results to JSON file, appending if file exists"""
    
    if os.path.exists(out_path):
        try:
            # Load existing data
            with open(out_path, 'r') as f:
                existing_data = json.load(f)
            
            # Ensure existing_data is a list
            if not isinstance(existing_data, list):
                existing_data = [existing_data]
            
            # Append new results
            existing_data.extend(results)
            
        except json.JSONDecodeError:
            # If existing file is corrupted, start fresh
            print("Warning: Existing JSON file corrupted, starting fresh")
            existing_data = results
        
        # Save combined data
        with open(out_path, 'w') as f:
            json.dump(existing_data, f, indent=2)
    else:
        # Create new file
        with open(out_path, 'w') as f:
            json.dump(results, f, indent=2)
            
# out_path = "/kaggle/working/iterative_featuresselection.json"
# save_results_to_json(results, out_path)

In [11]:
# print("Training for Iteration 1 (shap=True)...")
# iter_1 = iterative_featureselection(
#     model=model,
#     X=X,
#     y=y,
#     split_fn=lambda X:split_rollingwindow(X=X, n_splits = 10, train_ratio=0.5),
#     # split_fn=lambda X:split_overlapwindow(X, 50000, 50000, 25000),
#     scorers = scorers,
#     drop_fraction=0.1,
#     min_features=10,
#     use_shap = True
# )
# out_path = "/kaggle/working/iterative_featuresselection.json"
# save_results_to_json(iter_1, out_path)

In [12]:
print("Training for Iteration 2 (shap = False)...")
iter_2 = iterative_featureselection(
    model=model,
    X=X,
    y=y,
    # split_fn=lambda X:split_overlapwindow(X, 50000, 50000, 25000),
    split_fn=lambda X:split_rollingwindow(X=X, n_splits = 10, train_ratio=0.5),
    scorers = scorers,
    drop_fraction=0.1,
    min_features=10,
    use_shap = False
)
out_path = "/kaggle/working/iterative_featuresselection.json"
save_results_to_json(iter_2, out_path)

Training for Iteration 2 (shap = False)...

[+] Iteration 1 - 871 features
    → PEARSON: 0.0655 | MSE: 1.7711 | MAE: 0.8491

[+] Iteration 2 - 784 features
    → PEARSON: 0.0647 | MSE: 1.8442 | MAE: 0.8674

[+] Iteration 3 - 706 features
    → PEARSON: 0.0705 | MSE: 1.9058 | MAE: 0.8805

[+] Iteration 4 - 636 features
    → PEARSON: 0.0641 | MSE: 1.8127 | MAE: 0.8572

[+] Iteration 5 - 573 features
    → PEARSON: 0.0606 | MSE: 1.7308 | MAE: 0.8560

[+] Iteration 6 - 516 features
    → PEARSON: 0.0672 | MSE: 1.8002 | MAE: 0.8849

[+] Iteration 7 - 465 features
    → PEARSON: 0.0690 | MSE: 1.8200 | MAE: 0.8726

[+] Iteration 8 - 419 features
    → PEARSON: 0.0615 | MSE: 2.0603 | MAE: 0.9047

[+] Iteration 9 - 378 features
    → PEARSON: 0.0658 | MSE: 2.0475 | MAE: 0.9332

[+] Iteration 10 - 341 features
    → PEARSON: 0.0688 | MSE: 2.0705 | MAE: 0.9136

[+] Iteration 11 - 307 features
    → PEARSON: 0.0691 | MSE: 2.1278 | MAE: 0.9290

[+] Iteration 12 - 277 features
    → PEARSON: 0.063

# Current Results

In [13]:
with open("/kaggle/working/iterative_featuresselection.json", "r") as f:
    current_results = json.load(f)

current_results_df = pl.DataFrame(current_results)
current_results_df

ID,params,num_features,features,scores_pearson_mean,scores_mse_mean,scores_mae_mean
str,struct[4],i64,list[str],f64,f64,f64
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",871,"[""volume"", ""X1"", … ""bidask_size""]",0.065463,1.771077,0.849132
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",784,"[""volume"", ""X1"", … ""bidask_size""]",0.064749,1.844201,0.867401
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",706,"[""X8"", ""X9"", … ""bidask_size""]",0.070511,1.905795,0.880457
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",636,"[""X15"", ""X16"", … ""bidask_size""]",0.064143,1.812719,0.857174
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",573,"[""X17"", ""X18"", … ""timestamp""]",0.060635,1.73081,0.856017
…,…,…,…,…,…,…
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",59,"[""X426"", ""X427"", … ""timestamp""]",0.018354,2.118647,0.977204
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",49,"[""X528"", ""X533"", … ""timestamp""]",-0.003859,1.992247,0.922619
"""4603eaa462664e56b89978d95cd8f6…","{0.1,10,false,true}",39,"[""X535"", ""X536"", … ""timestamp""]",-0.004086,2.200124,0.966154
